# Análise de qualidade dos dados

Aqui faremos a etapa de análise de qualidade de cada atributo do conjunto de dados. Faremos isso utilizando consultas SQL baseadas nas características de cada atributo registradas no catálogo de dados.

## 1. Project_name
O nome do projeto é um texto livre, porém não pode ser vazio.

Vamos verificar se há linhas em que o nome do projeto seja nulo.

In [0]:
%sql
SELECT count(1) AS `Nomes de projeto vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_name IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 2. ID
O processo de ETL já assegurou que o ID seja sempre um número inteiro.

Além disso, ele deve ser único. Vamos verificar.

In [0]:
%sql
SELECT ID AS `ID repetido`, count(ID) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
GROUP BY ID
HAVING count(ID) > 1

Encontramos três IDs que se repetem na tabela. Vamos olhar para esses dados.

In [0]:
%sql
SELECT *
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE ID IN (1103, 365, 863)
ORDER BY ID ASC

Vamos que os dados não são idênticos, porém a similaridade dos dados indica que alguns dos elementos duplicados possam estar relacionados, ou até mesmo ser uma continuação do projeto.

Não conhecemos plenamente a semântica da repetição do ID, por isso não alteraremos os dados.

## 3. Country
O valor do campo pode ser um texto livre indicando um ou mais países ou regiões. Vamos apenas verificar se há linhas vazias.

In [0]:
%sql
SELECT count(1) as `Países vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Country IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 4. Partners
O valor do campo pode ser um texto livre indicando as empresas envolvidas no projeto. Vamos apenas verificar se há linhas vazias.

In [0]:
%sql
SELECT count(1) as `Parceiros vazios`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Partners IS NULL

Não há campos vazios. Dessa forma, não se verificou problemas na coluna.

## 5. Project_type
O tipo de projeto deve ser um dos valores categóricos permitidos. Vamos verificar.

In [0]:
%sql
SELECT Project_type, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_type NOT IN ('Full chain', 'Capture', 'Transport', 'Storage', 'T&S', 'CCU')
GROUP BY Project_type

Identificamos oito registros em que o tipo "Full Chain" foi preenchido de forma irregular, com o C maiúsculo. Vamos corrigir esses valores.

In [0]:
%sql
UPDATE dt0051_dev.treinamento.iea_ccus_projects
SET Project_type = 'Full chain'
WHERE Project_type == 'Full Chain'

A saída mostra que oito linhas foram alteradas. Vamos refazer a consulta para nos certificar de que o problema foi corrigido.

In [0]:
%sql
SELECT Project_type, count(1) AS Quantidade
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Project_type NOT IN ('Full chain', 'Capture', 'Transport', 'Storage', 'T&S', 'CCU')
GROUP BY Project_type

Nenhum resultado foi retornado. Sendo assim, não há mais nenhum problema com a coluna Project_type.

## 6. Announcement
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

Ele deve ser também um ano válido, não pode ser futuro, e não pode ser num passado muito distante. Vamos verificar esses limites, escolhendo como margem mínima para a avaliação o ano de 1950, época em que a indústria de petróleo ainda começava a crescer mundialmente.

In [0]:
%sql
SELECT count(1) AS `Datas de anúncio fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Announcement < 1950 OR Announcement > 2024

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 7. FID
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data de anúncio do projeto.

In [0]:
%sql
SELECT count(1) AS `FIDs fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE FID > 2124 OR FID < Announcement

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 8. Operation
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data final de investimento do projeto.

In [0]:
%sql
SELECT count(1) AS `Datas de operação fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Operation > 2124 OR Operation < FID

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.

## 9. Suspension_decommissioning
O processo de ETL já assegurou que o valor desta coluna seja sempre um número inteiro.

A data pode ser futura, porém não pode ser um futuro muito distante. Daremos uma margem de 100 anos no futuro. Além disso, não faz sentido que seja anterior à data de início de operação do projeto.

In [0]:
%sql
SELECT count(1) AS `Datas de descomissionamento fora da margem`
FROM dt0051_dev.treinamento.iea_ccus_projects
WHERE Suspension_decommissioning > 2124 OR Suspension_decommissioning < Operation

Nenhuma data foi encontrada fora da margem definida. Com isso, não há problemas de qualidade a resolver.